In [ ]:
# Load Drive Helper
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import các thư viên cần thiết
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
DATASET_PATH = '/content/drive/My Drive/NT204/dataset/'
TRAINSET_PATH = DATASET_PATH + 'train_set.csv'
TESTSET_PATH = DATASET_PATH + 'test_set.csv'

# Tạo thư mục để tải dataset
if not os.path.exists(DATASET_PATH):
  os.makedirs(DATASET_PATH)

'''
Tải tập train (train_set.csv) và tập test (test_set.csv) vào thư mục /content/drive/My Drive/NT204/dataset/ trên drive.
Lưu ý: Đặt đúng tên các file.
- Tải các tập dữ liệu tại: https://drive.google.com/drive/folders/1nYyDqbmichotvI9eSOrKo-yRwtLUy_iM?usp=sharing
'''

'\nTải tập train (train_set.csv) và tập test (test_set.csv) vào thư mục /content/drive/My Drive/NT204/dataset/ trên drive.\nLưu ý: Đặt đúng tên các file.\n- Tải các tập dữ liệu tại: https://drive.google.com/drive/folders/1nYyDqbmichotvI9eSOrKo-yRwtLUy_iM?usp=sharing\n'

In [ ]:
# Tiền xử lý dữ liệu
lable_encoder = preprocessing.LabelEncoder()

# Tập train
data_train = pd.read_csv(TRAINSET_PATH)

x_train, y_train = data_train.iloc[:, :-1], data_train.iloc[:, [-1]]
y_train = y_train.apply(lable_encoder.fit_transform)
x_train, y_train = x_train.to_numpy(), y_train.to_numpy()

# Lấy số thuộc tính và số lớp
NUM_FEATURES = x_train.shape[1] # Số thuộc tính bằng số chiều (số cột của x_train hoặc x_test)
NUM_CLASSES = np.unique(y_train).size # Số class bằng tổng số các giá trị khác nhau của y_train

# one-hot encoding label
y_train = tf.keras.utils.to_categorical(y_train)

In [ ]:
print("Number of features: %s" %NUM_FEATURES)
print("Number of classes: %s" %NUM_CLASSES)

Number of features: 42
Number of classes: 11


In [ ]:
model = Sequential([
    # Thêm một tầng LSTM với 50 đơn vị. 'input_shape' chỉ ra kích thước của dữ liệu đầu vào.
    #'return_sequences=True' để chồng các tầng LSTM; nó cho biết output của tầng này
    # sẽ có cấu trúc chuỗi để đầu vào của tầng LSTM tiếp theo có thể hiểu được.
    LSTM(50, input_shape=(x_train.shape[1], 1), return_sequences=True),
    # Thêm một tầng Dropout để giảm hiện tượng overfitting. Dropout sẽ ngẫu nhiên loại bỏ (tức là thiết lập thành zero)
    # một phần các đặc trưng đầu vào trong quá trình đào tạo với tỷ lệ là 0.2.
    Dropout(0.2),
    # Thêm một tầng LSTM thứ hai với 25 đơn vị. Không cần 'return_sequences' ở đây vì đây là tầng LSTM cuối cùng
    # và chỉ cần trả về output cuối cùng cho tầng Dense tiếp theo.
    LSTM(25),
     # Thêm một tầng Dense. Tầng này là một tầng kết nối đầy đủ với 50 nơ-ron và sử dụng hàm kích hoạt ReLU.
    Dense(50, activation='relu'),
     # Tầng output cuối cùng là một tầng Dense với số lượng nơ-ron bằng số lượng class trong dữ liệu đầu ra (y_train).
    # Sử dụng hàm kích hoạt 'softmax' để chuyển đổi các giá trị logits thành xác suất cho mỗi class.
    Dense(y_train.shape[1], activation='softmax')
])

# Compile mô hình
# Compile mô hình sử dụng:
# - 'adam' làm bộ tối ưu hóa, một biến thể của gradient descent có điều chỉnh tỷ lệ học tự động.
# - 'categorical_crossentropy' làm hàm mất mát, phù hợp với các bài toán phân loại đa lớp khi đầu ra là one-hot encoded.
# - 'accuracy' là chỉ số được theo dõi trong quá trình đào tạo và đánh giá mô hình.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Xem thông tin của mô hình
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 42, 50)            10400     
                                                                 
 dropout (Dropout)           (None, 42, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 25)                7600      
                                                                 
 dense (Dense)               (None, 50)                1300      
                                                                 
 dense_1 (Dense)             (None, 11)                561       
                                                                 
Total params: 19861 (77.58 KB)
Trainable params: 19861 (77.58 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Lưu mô hình
model.save('/content/drive/My Drive/NT204/model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Huấn luyện mô hình
# Sử dụng dữ liệu đã chuẩn bị, huấn luyện mô hình với 30 epochs.
# Batch size là 64, có nghĩa là mỗi lần cập nhật trọng số sử dụng 64 mẫu.
# Validation split là 0.2, tức là 20% dữ liệu huấn luyện được sử dụng để đánh giá mô hình trong quá trình huấn luyện.
model.fit(x_train, y_train, epochs=30, batch_size=64, validation_split=0.2)

Epoch 1/30
22226/22226 [==============================] - 198s 9ms/step - loss: 0.1195 - accuracy: 0.9603 - val_loss: 0.0912 - val_accuracy: 0.9670
Epoch 2/30
22226/22226 [==============================] - 190s 9ms/step - loss: 0.0886 - accuracy: 0.9678 - val_loss: 0.0810 - val_accuracy: 0.9705
Epoch 3/30
22226/22226 [==============================] - 190s 9ms/step - loss: 0.0788 - accuracy: 0.9702 - val_loss: 0.0732 - val_accuracy: 0.9715
Epoch 4/30
22226/22226 [==============================] - 189s 9ms/step - loss: 0.0716 - accuracy: 0.9723 - val_loss: 0.0648 - val_accuracy: 0.9744
Epoch 5/30
22226/22226 [==============================] - 190s 9ms/step - loss: 0.0676 - accuracy: 0.9735 - val_loss: 0.0647 - val_accuracy: 0.9743
Epoch 6/30
22226/22226 [==============================] - 189s 9ms/step - loss: 0.0639 - accuracy: 0.9747 - val_loss: 0.0609 - val_accuracy: 0.9758
Epoch 7/30
22226/22226 [==============================] - 188s 8ms/step - loss: 0.0621 - accuracy: 0.9754 - val_

In [ ]:
# Tiền xử lý dữ liệu kiểm thử
data_test = pd.read_csv(TESTSET_PATH)
x_test, y_test = data_test.iloc[:, :-1], data_test.iloc[:, [-1]]
y_test = y_test.apply(lable_encoder.transform)  # Sử dụng transform, không phải fit_transform
x_test, y_test = x_test.to_numpy(), y_test.to_numpy()

# One-hot encoding cho y_test để phù hợp với cách mô hình được huấn luyện.
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
# Đánh giá mô hình
# Đánh giá mô hình trên dữ liệu kiểm thử và in ra loss và accuracy.
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Dự đoán và tính các chỉ số
# Sử dụng mô hình để dự đoán dữ liệu kiểm thử.
predictions = model.predict(x_test)
# Lấy chỉ số của lớp có xác suất cao nhất từ kết quả dự đoán.
predicted_classes = np.argmax(predictions, axis=1)
# Lấy chỉ số lớp từ y_test.
true_classes = np.argmax(y_test, axis=1)

# Tính Precision, Recall và F1-Score
# Tính toán các chỉ số đánh giá chính cho mô hình.
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print(f"Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

23813/23813 [==============================] - 99s 4ms/step - loss: 0.0521 - accuracy: 0.9791
Loss: 0.052093133330345154, Accuracy: 0.9790542125701904
23813/23813 [==============================] - 79s 3ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.9774788410129246, Recall: 0.9790542180928197, F1-Score: 0.9757508171642557


In [ ]:
# Kiểm tra mô hình với tập test
preds = model.predict(x_test, batch_size=1024)

745/745 [==============================] - 3s 4ms/step


In [ ]:
# Tính toán kết quả
# Chuyển y_test từ one-hot về chỉ số lớp để tính toán các chỉ số.
y_test = np.argmax(y_test, axis=1)

y_pred = np.argmax(preds, axis=1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print('Accuracy: %s\nPrecision: %s\nRecal: %s\nF1-Score: %s' %(accuracy, precision, recall, f1))

Accuracy: 0.9790542180928197
Precision: 0.9774788410129246
Recal: 0.9790542180928197
F1-Score: 0.9757508171642557
